In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install tfa-nightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.3/620.3 kB 14.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/cnn21_base_model_PL_fold_4-12-0.8685-0.8758.zip'

Archive:  /content/drive/MyDrive/cnn21_base_model_PL_fold_4-12-0.8685-0.8758.zip
  inflating: cnn21_base_model_PL_fold_4-12-0.8685-0.8758.h5  


In [ ]:
source1 = "/content/drive/MyDrive/DepthwiseConv1D.py"
source2 = "/content/drive/MyDrive/create_images1.py"
source3= "/content/drive/MyDrive/X_test4.npy"
source4 = "/content/drive/MyDrive/my_classes.py"
source5 = "/content/drive/MyDrive/signal_processing2.py"
source6 = "/content/drive/MyDrive/cnn21_base_model_PL_fold_4-12-0.8685-0.8758.h5"

destination = "/content/"
!cp $source1 $destination
print("copy1 done!")
!cp $source2 $destination
print("copy2 done!")
!cp $source3 $destination
print("copy3 done!")
!cp $source4 $destination
print("copy4 done!")
!cp $source5 $destination
print("copy5 done!")
!cp $source6 $destination
print("copy6 done!")

copy1 done!
copy2 done!
copy3 done!
copy4 done!
copy5 done!
copy6 done!


In [ ]:
import h5py

In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
import tensorflow

In [ ]:
print(tensorflow.__version__)

2.15.0


In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow-2.6.0-cp37-cp37m-manylinux2010_x86_64.whl

ERROR: tensorflow-2.6.0-cp37-cp37m-manylinux2010_x86_64.whl is not a supported wheel on this platform.


In [ ]:
import os

import tensorflow as tf
import pandas as pd
from my_classes import CWTDataGen
from my_classes import AdditionalMetrics, SaveModels, CustomModel, ConfusionMatrixMetric
import numpy as np
import statistics
import random
import tqdm.auto as tqdm

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import LSTM, RNN, Dropout, ReLU, Flatten, Dense, Add, multiply, Reshape
from tensorflow.keras.layers import Bidirectional, LayerNormalization, Conv1D, MaxPool1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Lambda, Input, BatchNormalization, Activation, Concatenate, SeparableConv1D
from tensorflow_addons.rnn import LayerNormLSTMCell
from DepthwiseConv1D import DepthwiseConv1D
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from tensorflow import experimental as tfe
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.signal import resample

import signal_processing2
from signal_processing2 import apply_filter, resample_timescale, normalise_length_zero_pad

print("imports done!")
def create_filtered_signals_cambell( sig, length_norm=9000):

        # Apply filter
        filtered = apply_filter(sig)
        # Resample
        filtered = resample_timescale(filtered, 250, 300)
        # Normalise signal length
        filtered = normalise_length_zero_pad(filtered, length_norm)

        # Expand dims to fit into CNN
        filtered = np.expand_dims(filtered, 1)
        return filtered


for fold in [4]:
    print('fold: ' + str(fold))
    if fold == 1:
        model = tf.keras.models.load_model('cnn21_base_model_PL_fold_1-11-0.8919-0.8938.h5',  custom_objects={"CustomModel": CustomModel,
                                                                               "ConfusionMatrixMetric": ConfusionMatrixMetric,
                                                                               "DepthwiseConv1D": DepthwiseConv1D})
    elif fold == 2:
        model = tf.keras.models.load_model('cnn21_base_model_PL_fold_2-07-0.8422-0.8684.h5',  custom_objects={"CustomModel": CustomModel,
                                                                               "ConfusionMatrixMetric": ConfusionMatrixMetric,
                                                                               "DepthwiseConv1D": DepthwiseConv1D})
    elif fold == 3:
        model = tf.keras.models.load_model('cnn21_base_model_PL_fold_3-10-0.8662-0.8678.h5',  custom_objects={"CustomModel": CustomModel,
                                                                               "ConfusionMatrixMetric": ConfusionMatrixMetric,
                                                                               "DepthwiseConv1D": DepthwiseConv1D})
    elif fold == 4:
        model = tf.keras.models.load_model('cnn21_base_model_PL_fold_4-12-0.8685-0.8758.h5', custom_objects={"CustomModel": CustomModel, "ConfusionMatrixMetric": ConfusionMatrixMetric, "DepthwiseConv1D": DepthwiseConv1D})
    else:
        model = tf.keras.models.load_model('cnn21_base_model_PL_fold_5-14-0.8922-0.8774.h5',  custom_objects={"CustomModel": CustomModel,
                                                                               "ConfusionMatrixMetric": ConfusionMatrixMetric,
                                                                               "DepthwiseConv1D": DepthwiseConv1D})


    print("folds done!")




    modelPredictProbAll = []



    PredictionAll = []
    PredictionProbAll = []
    LossAll=[]
    dirsAll = []

    print("variables done!")


    modelNN = tf.keras.models.load_model('D:/NewJimmyCodeBackUPOnD/Cambell/50 Pretrained NNs/'+files0NN)
    print("model defination done!")
    for root, dirs, files in os.walk('D:/NewJimmyCodeBackUPOnD/Cambell/Data'):
        for dirs0 in dirs:
            print(dirs0)
            dirsAll.append(dirs0)
            fin = open(os.path.join(root,dirs0)+'/ecg.dat', 'rb')
            a = np.fromfile(fin, dtype=np.dtype('<u2'))
            """
                    X=[]
                    for x in a:
                     #  print(x)
                        X.append(x)
                        new_sample = np.asarray(X)
                      # print(X)
                        new_sample_flt = np.squeeze(create_filtered_signals_cambell(new_sample))


                        new_sample_flt_downsamp = resample(new_sample_flt, 4500)
                        new_sample_flt_downsamp = np.expand_dims(new_sample_flt_downsamp, 1)

                        PredictionProbAll.append(model.predict(new_sample_flt_downsamp))
            """
            new_sample_flt = np.squeeze(create_filtered_signals_cambell(a))
            new_sample_flt_downsamp = resample(new_sample_flt, 4500)
          #  new_sample_flt_downsamp = new_sample_flt[::2] # Decimate more efficient to downsample by integer factors
            new_sample_flt_downsamp = np.expand_dims(np.expand_dims(new_sample_flt_downsamp, 1), 0) # Make sure shape is Batch_size, N_timesteps, N_channels

            PredictionProbAll.append(model.predict(new_sample_flt_downsamp))







In [ ]:
import os

import tensorflow as tf
import pandas as pd
from my_classes import CWTDataGen
from my_classes import AdditionalMetrics, SaveModels, CustomModel, ConfusionMatrixMetric
import numpy as np
import statistics
import random
import tqdm.auto as tqdm

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import LSTM, RNN, Dropout, ReLU, Flatten, Dense, Add, multiply, Reshape
from tensorflow.keras.layers import Bidirectional, LayerNormalization, Conv1D, MaxPool1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Lambda, Input, BatchNormalization, Activation, Concatenate, SeparableConv1D
from tensorflow_addons.rnn import LayerNormLSTMCell
from DepthwiseConv1D import DepthwiseConv1D
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from tensorflow import experimental as tfe
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.signal import resample

import signal_processing2
from signal_processing2 import apply_filter, resample_timescale, normalise_length_zero_pad

print("imports done!")
def create_filtered_signals_cambell( sig, length_norm=9000):

        # Apply filter
        filtered = apply_filter(sig)
        # Resample
        filtered = resample_timescale(filtered, 250, 300)
        # Normalise signal length
        filtered = normalise_length_zero_pad(filtered, length_norm)

        # Expand dims to fit into CNN
        filtered = np.expand_dims(filtered, 1)
        return filtered
model = tf.keras.models.load_model('cnn21_base_model_PL_fold_4-12-0.8685-0.8758.h5',  custom_objects={"CustomModel": CustomModel,
                                                                               "ConfusionMatrixMetric": ConfusionMatrixMetric,
                                                                               "DepthwiseConv1D": DepthwiseConv1D})


print("model done!")

In [ ]:
import pandas as pd
dfs = pd.read_excel('D:/NewJimmyCodeBackUPOnD/Cambell/Label/Cardiologist Interpretations Consensus.xlsx')

In [ ]:
UID = dfs.iloc[1:,0].tolist()

IdxSorted = []
UIDInUID = []
IdxInUID = []
c = 0
for idx in UID:
    if idx in dirsAll:
        IdxSorted.append(dirsAll.index(idx))
        UIDInUID.append(idx)
        IdxInUID.append(c+1)
    c+=1

IdxInUID = np.asarray(IdxInUID)
IdxSorted = np.asarray(IdxSorted)

In [ ]:
CamLabels = dfs.iloc[IdxInUID,1].tolist()

In [ ]:
Ref = []
missingInd = []
for i in range(len(CamLabels)):
    if CamLabels[i]=='AFIB':
        Ref.append(1)
    elif CamLabels[i]=='NSR':
        Ref.append(2)
    elif CamLabels[i]=='NSR/ OTHER':
        Ref.append(3)
    elif CamLabels[i]=='OTHER':
        Ref.append(3)
    elif CamLabels[i]=='Too noisy':
        Ref.append(4)
    else:
        Ref.append(5)
        missingInd.append(i)

Ref = np.asarray(Ref)
missingInd = np.asarray(missingInd)

In [ ]:
PredictionProbAll = np.asarray(PredictionProbAll)
PredictionProbAllSq = np.squeeze(PredictionProbAll)
PredictionProbAllSqSorted = PredictionProbAllSq[IdxSorted]

In [ ]:
Predictions = np.argmax(PredictionProbAllSqSorted,axis=1)+1

In [ ]:
from sklearn.metrics import classification_report
target_names=['AF','NSR','Oth','TN']
print(classification_report((Ref),Predictions,target_names=target_names, digits=4))

              precision    recall  f1-score   support

          AF     0.8756    0.6815    0.7664       248
         NSR     0.9416    0.5070    0.6591       286
         Oth     0.2780    0.7808    0.4101        73
          TN     0.1548    0.4483    0.2301        29

    accuracy                         0.6038       636
   macro avg     0.5625    0.6044    0.5164       636
weighted avg     0.8038    0.6038    0.6528       636

